In [1]:
import numpy as np
import pandas as pd
import ta
import yfinance as yf
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime
import ccxt
from tqdm import tqdm
tqdm.pandas()

%config InlineBackend.figure_format = 'retina'

In [ ]:
# test net futures api keys
API_KEY = ''
SECRET_KEY = ''

# Initialize Binance exchange with API keys
exchange = ccxt.binance({
    'apiKey': API_KEY,
    'secret': SECRET_KEY,
})
exchange.set_sandbox_mode(True)

In [4]:
def fetch_binance_data(symbol='BTC/USDT', timeframe='1h', since=None, limit=None):
    """
    Fetch historical data from Binance.

    Parameters:
    - symbol (str): Trading pair symbol (e.g., 'BTC/USDT').
    - timeframe (str): Timeframe for the candles (e.g., '1m', '5m', '1h', '1d').
    - since (int): Unix timestamp in milliseconds to start fetching data from.
    - limit (int): Number of data points to fetch (max 1000 per request).

    Returns:
    - df (pd.DataFrame): DataFrame containing the historical data.
    """
    # Initialize Binance exchange
    exchange = ccxt.binance()

    # Fetch OHLCV data
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit)

    # Convert to DataFrame
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])

    # Convert timestamp to datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

# Prepare data & Feature engineer

In [75]:
def feature_engineering(df, spans=[30,50], rsi_window=9):
    df = df.copy()
    for w, s in zip(spans, ['Fast', 'Slow']):
        # df[f'SMA_{s}'] = df['Close'].rolling(window=w).mean()
        df[f'EMA_{s}'] = df['Close'].ewm(span=w, adjust=False).mean()
        # df[f'WMA_{s}'] = df['Close'].rolling(window=w).apply(lambda x: np.dot(x, np.arange(1, w+1)) / x.sum())
    
    df['MACD'] = ta.trend.macd(df['Close'], window_slow=26, window_fast=12)
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'], window_slow=26, window_fast=12, window_sign=9)
        
    df['Bollinger_Upper'] = ta.volatility.bollinger_hband(df['Close'], window=15, window_dev=2)
    df['Bollinger_Lower'] = ta.volatility.bollinger_lband(df['Close'], window=15, window_dev=2)
    
    df['ATR'] = ta.volatility.average_true_range(df['High'], df['Low'], df['Close'], window=7)
    
    # Calculate RSI
    df['RSI'] = ta.momentum.rsi(df['Close'], window=rsi_window)
    
    # df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    # df['VWAP'] = ta.volume.volume_weighted_average_price(df['High'], df['Low'], df['Close'], df['Volume'], window=rsi_window)

    df.dropna(inplace=True)
    
    return df

In [76]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_Fast values are below EMA_Slow values
    if all(relevant_rows["EMA_Fast"] < relevant_rows["EMA_Slow"]):
        return 1
    elif all(relevant_rows["EMA_Fast"] > relevant_rows["EMA_Slow"]):
        return 2
    else:
        return 0

In [85]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['Bollinger_Lower'][current_candle]
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['Bollinger_Upper'][current_candle]
        ):
    
            return 1
    return 0

In [78]:
def pointpos(x):
    if x['TotalSignal']==2:
        return x['Low']-1e-3
    elif x['TotalSignal']==1:
        return x['High']+1e-3
    else:
        return np.nan

In [196]:
# df = pd.read_csv('SOLUSDT-5m-2023-09.csv').iloc[:,:6]
# df.columns = ['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume']
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# df.set_index('timestamp', inplace=True)

In [197]:
df = fetch_binance_data(symbol='XRP/USDT', timeframe='5m', limit=None)
df = feature_engineering(df)

df=df[:-1]
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

100%|██████████| 466/466 [00:00<00:00, 1601.25it/s]


In [198]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['Bollinger_Lower'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['Bollinger_Upper'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_Fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_Fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_Slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_Slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

In [211]:
from backtesting import Strategy
from backtesting import Backtest

def SIGNAL():
    return df.TotalSignal

class MyStrat(Strategy):
    mysize = 30000
    slcoef = 1.1
    TPSLRatio = 2
    rsi_length = 16
    
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        # if len(self.trades)>0:
        #     if self.trades[-1].is_long and self.data.RSI[-1]>=90:
        #         self.trades[-1].close()
        #     elif self.trades[-1].is_short and self.data.RSI[-1]<=10:
        #         self.trades[-1].close()
        
        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
        
        elif self.signal1==1 and len(self.trades)==0:         
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df.set_index('timestamp'), MyStrat, cash=500, margin=1/30, commission=.00075)

In [212]:
bt.run()

/Users/pax/miniconda3/envs/price/lib/python3.10/site-packages/backtesting/_stats.py:122: RuntimeWarning:

divide by zero encountered in scalar divide



Start                     2024-08-03 17:45:00
End                       2024-08-05 08:30:00
Duration                      1 days 14:45:00
Exposure Time [%]                    2.360515
Equity Final [$]                   739.458851
Equity Peak [$]                    739.458851
Return [%]                           47.89177
Buy & Hold Return [%]               -15.68309
Return (Ann.) [%]         4748111363667007...
Volatility (Ann.) [%]     2435720342072127...
Sharpe Ratio                              0.0
Sortino Ratio                             inf
Calmar Ratio              4469012341834906...
Max. Drawdown [%]                  -10.624521
Avg. Drawdown [%]                   -6.819282
Max. Drawdown Duration        0 days 00:25:00
Avg. Drawdown Duration        0 days 00:18:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                        1.66485
Worst Trade [%]                       1.66485
Avg. Trade [%]                    

In [93]:
exchange = ccxt.binance({
    'apiKey': API_KEY,
    'secret': SECRET_KEY,
    'enableRateLimit': True
})
exchange.set_sandbox_mode(True)

In [117]:
class MyStrat(Strategy):
    # mysize = 3000
    # slcoef = 1.1
    # TPSLRatio = 1.5
    
    def init(self, mysize=3000, slcoef=1.1, TPSLRatio=1.5):
        super().init()
        self.signal1 = self.I(SIGNAL)
        self.mysize = mysize
        self.slcoef = slcoef
        self.TPSLRatio = TPSLRatio

    def next(self):
        super().next()
        self.slatr = self.slcoef * self.data.ATR[-1]
        # TPSLRatio = self.TPSLRatio

        if self.signal1[-1] == 2 and len(self.trades) == 0:
            sl1 = self.data.Close[-1] - self.slatr
            tp1 = self.data.Close[-1] + self.slatr * self.TPSLRatio
            # self.place_order('buy', sl1, tp1)

        elif self.signal1[-1] == 1 and len(self.trades) == 0:
            sl1 = self.data.Close[-1] + self.slatr
            tp1 = self.data.Close[-1] - self.slatr * self.TPSLRatio
            # self.place_order('sell', sl1, tp1)
    
    # def place_order(self, side, stop_loss, take_profit):
    #     symbol = 'BTC/USDT'
    #     if side == 'buy':
    #         order = exchange.create_order(symbol, 'market', 'buy', self.mysize)
    #     elif side == 'sell':
    #         order = exchange.create_order(symbol, 'market', 'sell', self.mysize)

In [118]:
bt = Backtest(df.set_index('timestamp'), MyStrat, cash=250, margin=1/30)

In [119]:
bt.run()

Start                     2024-08-04 10:23:00
End                       2024-08-04 18:08:00
Duration                      0 days 07:45:00
Exposure Time [%]                         0.0
Equity Final [$]                        250.0
Equity Peak [$]                         250.0
Return [%]                                0.0
Buy & Hold Return [%]               -4.403435
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [ ]:
while True:
    df = fetch_binance_data(symbol='BTC/USDT', timeframe='1h', limit=100)
    df = feature_engineering(df)
    
    # Run the strategy
    bt = Backtest(df, MyStrat, cash=250, margin=1/30)
    bt.run()
    
    # Sleep for the duration of the timeframe
    time.sleep(3600)  # Sleep for 1 hour if timeframe is 1h